In [1]:
import numpy as np
import pandas as pd
import glob
import os
os.getcwd()
os.chdir('/project/time_series')
from load_data_custom import load_data_using_multi_process
from make_roc_curve import make_roc_curve
os.chdir('/project/time_series/cchc0427')
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from model import model_lgbm
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
import lazy_loader as lazy
samplerate = lazy.load("samplerate")
resampy = lazy.load("resampy")


In [2]:
import glob
fan_path = '/data/time_series/fan/_6'
len_fan_path = len(glob.glob( fan_path + '/*/*/*' ))

# fan_files_6 = glob.glob(fan_path + '/_6/*/*/*')
# fan_files6 = glob.glob(fan_path + '/6/*/*/*')
fan_files_6 = glob.glob( fan_path + '/*/*/*')

In [3]:
target_lst = []
for i in range(1, 16001):
    if ( i < 40 ) or ( i > 13000):
        continue
    if 16000 % i == 0 :
        target_lst.append(i)

target_lst.append(10000)
target_lst.append(13000)

In [4]:
target_lst

[40,
 50,
 64,
 80,
 100,
 125,
 128,
 160,
 200,
 250,
 320,
 400,
 500,
 640,
 800,
 1000,
 1600,
 2000,
 3200,
 4000,
 8000,
 10000,
 13000]

In [1]:
result = []

for target_sr in target_lst:
    print(f'{target_sr}로 진행중입니다.')
    fan_result = load_data_using_multi_process(fan_files_6, sr = 16000, target_sr = target_sr, res_type = 'kaiser_best') # 데이터 target_sr로 부르기
    
    # librosa.resample하게되면 shape이 ( file수, 1, column수 ) squeeze함수는 shape에서 1인부분 제거 1의의미 모르겠음
    fan_squeeze_result = np.squeeze(fan_result) 
    
    fan_result_df = pd.DataFrame(fan_squeeze_result)
    
    data = fan_result_df.iloc[:, :-1]
    target = fan_result_df.iloc[:, -1]
    
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, shuffle=True, stratify=target, random_state=42)
    
    model = model_lgbm(fan_result_df) # using (model.py)
    
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)
    
    acc = accuracy_score(y_test, y_pred) 
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)    
    auc_score = roc_auc_score(y_test, y_pred_prob[ :, 1 ])
    
    result.append([ target_sr, round(acc, 3) ,round(recall, 3), round(precision, 3), round(f1, 3), round(auc_score, 3) ])

result_df = pd.DataFrame(result, columns = [ 'target_sr', 'acc', 'recall', 'precision', 'f1', 'auc_score'])
result_df.to_csv('/project/time_series/cchc0427/id_num/fanfile_6_target_sr.csv', index = False)


NameError: name 'target_lst' is not defined

In [10]:
target_sr_csv = pd.read_csv('fanfile_6_target_sr.csv')


In [11]:
target_sr_csv

,target_sr,acc,recall,precision,f1,auc_score
0,40,0.707,0.075,0.297,0.119,0.502
1,50,0.750,0.200,0.584,0.298,0.662
2,64,0.782,0.264,0.757,0.392,0.726
3,80,0.773,0.251,0.705,0.370,0.693
4,100,0.759,0.166,0.690,0.268,0.658
5,125,0.763,0.180,0.716,0.287,0.704
6,128,0.761,0.169,0.714,0.274,0.704
7,160,0.768,0.190,0.757,0.304,0.699
8,200,0.771,0.169,0.847,0.282,0.689
9,250,0.760,0.142,0.764,0.240,0.678


noise = 0 일때 : target_sr = 125, res_type = (soxr_lq, kaiser_fast) 에서 f1, auc 모두 제일높음   
noise = 6 일때 : target_sr = (128, 125), res_type = (soxr_lq) 에서 f1, auc 모두 제일높음      
noise = _6 일때 : target_sr = 64, res_type = (soxr_lq) 에서 f1, auc 모두 제일높음      

In [6]:
sampling_lst = ['abs_max(our_custom)',
                'kaiser_best',
                'kaiser_fast',
                'fft',
                'scipy',
                'polyphase',
                'linear',
                'zero_order_hold',
                'sinc_best',
                'sinc_medium' ,
                'sinc_fastest',
                'soxr_vhq',
                'soxr_hq',
                'soxr_mq',
                'soxr_lq',
                'soxr_hq',
                'soxr_qq']
target_sr = 64
result = []

for res_type in sampling_lst:
    print(f'{res_type}로 진행중입니다.')
    fan_result = fan_result = load_data_using_multi_process(fan_files_6, sr = 16000, target_sr = target_sr, res_type = res_type) # 데이터 target_sr로 부르기
    
    fan_result_df = pd.DataFrame(fan_result)
    
    data = fan_result_df.iloc[:, :-1]
    target = fan_result_df.iloc[:, -1]
    
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, shuffle=True, stratify=target, random_state=42)
    
    model = model_lgbm(fan_result_df) # using (model.py)
    
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)
    
    acc = accuracy_score(y_test, y_pred) 
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)    
    auc_score = roc_auc_score(y_test, y_pred_prob[ :, 1 ])
    
    result.append([ res_type, round(acc, 3) ,round(recall, 3), round(precision, 3), round(f1, 3), round(auc_score, 3) ])

result_df = pd.DataFrame(result, columns = [ 'res_type', 'acc', 'recall', 'precision', 'f1', 'auc_score'])
result_df.to_csv('/project/time_series/cchc0427/noise/fan_6_res_type.csv', index = False)


abs_max(our_custom)로 진행중입니다.

멀티 프로세싱을 시작합니다. 현재 코어 30개 사용 중. 
1.다른 사람과 동시에 돌리지 마세요.
2.함부로 진행중에 중단 하지마세요.
--- 걸린시간: 223.84290599822998 ---
normal : 4075
abnormal : 1475
scale_pos_weight is 2.76.
Model is ready to running.


/package/anaconda3.8/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


kaiser_best로 진행중입니다.

멀티 프로세싱을 시작합니다. 현재 코어 30개 사용 중. 
1.다른 사람과 동시에 돌리지 마세요.
2.함부로 진행중에 중단 하지마세요.
--- 걸린시간: 11.521564960479736 ---
normal : 4075
abnormal : 1475
scale_pos_weight is 2.76.
Model is ready to running.


/package/anaconda3.8/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


kaiser_fast로 진행중입니다.

멀티 프로세싱을 시작합니다. 현재 코어 30개 사용 중. 
1.다른 사람과 동시에 돌리지 마세요.
2.함부로 진행중에 중단 하지마세요.
--- 걸린시간: 6.11199951171875 ---
normal : 4075
abnormal : 1475
scale_pos_weight is 2.76.
Model is ready to running.


...

/package/anaconda3.8/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


soxr_hq로 진행중입니다.

멀티 프로세싱을 시작합니다. 현재 코어 30개 사용 중. 
1.다른 사람과 동시에 돌리지 마세요.
2.함부로 진행중에 중단 하지마세요.
--- 걸린시간: 2.6610958576202393 ---
normal : 4075
abnormal : 1475
scale_pos_weight is 2.76.
Model is ready to running.


/package/anaconda3.8/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


soxr_qq로 진행중입니다.

멀티 프로세싱을 시작합니다. 현재 코어 30개 사용 중. 
1.다른 사람과 동시에 돌리지 마세요.
2.함부로 진행중에 중단 하지마세요.
--- 걸린시간: 2.4742491245269775 ---
normal : 4075
abnormal : 1475
scale_pos_weight is 2.76.
Model is ready to running.


/package/anaconda3.8/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


In [4]:
res_type_6_csv = pd.read_csv('/project/time_series/cchc0427/noise/fanfile_6_res_type.csv')
res_type6_csv = pd.read_csv('/project/time_series/cchc0427/noise/fanfile6_res_type.csv')
res_type0_csv = pd.read_csv('/project/time_series/cchc0427/noise/fanfile0_res_type.csv')
res_type_csv

,res_type,acc,recall,precision,f1,auc_score
0,abs_max(our_custom),0.741,0.200,0.536,0.291,0.613
1,kaiser_best,0.782,0.264,0.757,0.392,0.726
2,kaiser_fast,0.783,0.275,0.750,0.402,0.726
3,fft,0.777,0.244,0.742,0.367,0.697
4,scipy,0.777,0.244,0.742,0.367,0.697
5,polyphase,0.768,0.231,0.694,0.346,0.707
6,linear,0.717,0.058,0.321,0.098,0.530
7,zero_order_hold,0.717,0.058,0.321,0.098,0.530
8,sinc_best,0.734,0.000,0.000,0.000,0.500
9,sinc_medium,0.734,0.000,0.000,0.000,0.500


In [6]:
res_type0_csv

,target_sr,acc,recall,precision,f1,auc_score
0,abs_max(our_custom),0.811,0.376,0.810,0.514,0.690
1,kaiser_best,0.842,0.444,0.923,0.600,0.867
2,kaiser_fast,0.853,0.485,0.929,0.637,0.873
3,fft,0.826,0.407,0.870,0.554,0.836
4,scipy,0.826,0.407,0.870,0.554,0.836
5,polyphase,0.834,0.431,0.888,0.580,0.853
6,linear,0.778,0.231,0.782,0.356,0.689
7,zero_order_hold,0.778,0.231,0.782,0.356,0.689
8,sinc_best,0.838,0.417,0.939,0.577,0.860
9,sinc_medium,0.837,0.431,0.907,0.584,0.867


In [9]:
target_sr_6_csv = pd.read_csv('/project/time_series/cchc0427/noise/fanfile_6_target_sr.csv')
target_sr6_csv = pd.read_csv('/project/time_series/cchc0427/noise/fanfile6_target_sr.csv')
target_sr0_csv = pd.read_csv('/project/time_series/cchc0427/noise/fanfile0_target_sr.csv')
target_sr_6_csv

,target_sr,acc,recall,precision,f1,auc_score
0,40,0.707,0.075,0.297,0.119,0.502
1,50,0.750,0.200,0.584,0.298,0.662
2,64,0.782,0.264,0.757,0.392,0.726
3,80,0.773,0.251,0.705,0.370,0.693
4,100,0.759,0.166,0.690,0.268,0.658
5,125,0.763,0.180,0.716,0.287,0.704
6,128,0.761,0.169,0.714,0.274,0.704
7,160,0.768,0.190,0.757,0.304,0.699
8,200,0.771,0.169,0.847,0.282,0.689
9,250,0.760,0.142,0.764,0.240,0.678


In [15]:
sr_result = pd.concat([target_sr_6_csv, target_sr0_csv, target_sr6_csv], axis = 1)
sr_result

,target_sr,acc,recall,precision,f1,auc_score,target_sr,acc,recall,precision,f1,auc_score,target_sr,acc,recall,precision,f1,auc_score
0,40,0.707,0.075,0.297,0.119,0.502,40,0.714,0.119,0.376,0.180,0.500,40,0.714,0.122,0.379,0.185,0.553
1,50,0.750,0.200,0.584,0.298,0.662,50,0.760,0.261,0.616,0.367,0.695,50,0.786,0.322,0.720,0.445,0.752
2,64,0.782,0.264,0.757,0.392,0.726,64,0.794,0.319,0.770,0.451,0.739,64,0.793,0.325,0.756,0.455,0.744
3,80,0.773,0.251,0.705,0.370,0.693,80,0.799,0.322,0.805,0.460,0.756,80,0.786,0.292,0.754,0.421,0.750
4,100,0.759,0.166,0.690,0.268,0.658,100,0.785,0.298,0.733,0.424,0.728,100,0.795,0.312,0.786,0.447,0.749
5,125,0.763,0.180,0.716,0.287,0.704,125,0.842,0.444,0.923,0.600,0.867,125,0.892,0.617,0.963,0.752,0.940
6,128,0.761,0.169,0.714,0.274,0.704,128,0.827,0.417,0.860,0.562,0.862,128,0.898,0.631,0.979,0.767,0.936
7,160,0.768,0.190,0.757,0.304,0.699,160,0.824,0.390,0.885,0.541,0.833,160,0.872,0.532,0.975,0.689,0.926
8,200,0.771,0.169,0.847,0.282,0.689,200,0.808,0.315,0.894,0.466,0.820,200,0.860,0.492,0.967,0.652,0.914
9,250,0.760,0.142,0.764,0.240,0.678,250,0.817,0.346,0.911,0.501,0.862,250,0.891,0.603,0.978,0.746,0.963


In [16]:
res_result = pd.concat([res_type_6_csv, res_type0_csv, res_type6_csv], axis = 1)
res_result

,res_type,acc,recall,precision,f1,auc_score,target_sr,acc,recall,precision,f1,auc_score,target_sr,acc,recall,precision,f1,auc_score
0,abs_max(our_custom),0.741,0.200,0.536,0.291,0.613,abs_max(our_custom),0.811,0.376,0.810,0.514,0.690,abs_max(our_custom),0.829,0.410,0.883,0.560,0.761
1,kaiser_best,0.782,0.264,0.757,0.392,0.726,kaiser_best,0.842,0.444,0.923,0.600,0.867,kaiser_best,0.892,0.617,0.963,0.752,0.940
2,kaiser_fast,0.783,0.275,0.750,0.402,0.726,kaiser_fast,0.853,0.485,0.929,0.637,0.873,kaiser_fast,0.888,0.603,0.962,0.742,0.941
3,fft,0.777,0.244,0.742,0.367,0.697,fft,0.826,0.407,0.870,0.554,0.836,fft,0.880,0.573,0.960,0.718,0.933
4,scipy,0.777,0.244,0.742,0.367,0.697,scipy,0.826,0.407,0.870,0.554,0.836,scipy,0.880,0.573,0.960,0.718,0.933
5,polyphase,0.768,0.231,0.694,0.346,0.707,polyphase,0.834,0.431,0.888,0.580,0.853,polyphase,0.888,0.600,0.967,0.741,0.932
6,linear,0.717,0.058,0.321,0.098,0.530,linear,0.778,0.231,0.782,0.356,0.689,linear,0.825,0.380,0.911,0.536,0.765
7,zero_order_hold,0.717,0.058,0.321,0.098,0.530,zero_order_hold,0.778,0.231,0.782,0.356,0.689,zero_order_hold,0.825,0.380,0.911,0.536,0.765
8,sinc_best,0.734,0.000,0.000,0.000,0.500,sinc_best,0.838,0.417,0.939,0.577,0.860,sinc_best,0.889,0.593,0.983,0.740,0.935
9,sinc_medium,0.734,0.000,0.000,0.000,0.500,sinc_medium,0.837,0.431,0.907,0.584,0.867,sinc_medium,0.893,0.610,0.978,0.752,0.941


# sr결과

- noise_6 : f1 = 64, auc = 64
- noise0 : 1,2등 둘다 (125, 128) 둘다해보기 
- noise6 : f1 = 128, auc = 250